# Input

In [1]:
target_column = "RecommendHiring"

In [2]:
# Parameters
target_column = "RecommendHiring"


# Constants

In [3]:
import os
import numpy as np
from sklearn.metrics import make_scorer

SCRIPT_DIR = os.path.join(os.getcwd(),)

SAVED_MODELS_PATH="./regression_models/saved_models"

DROPPED_LEXICAL_COLUMNS = [
    "Swear",
    "Numbers",
    "Inhibition",
    "Preceptual",
    "Anxiety",
    "Anger",
    "Sadness",
    "Work",
    "Articles",
    "Verbs",
    "Adverbs",
    "Prepositions",
    "Conjunctions",
    "Negations",
]

facial_features = [
    "average_inner_brow_height",
    "average_outer_brow_height",
    "eye_open",
    "inner_lip_height",
    "lip_corner_distance",
    "outer_lip_height",
    "smile",
    "pitch",
    "roll",
    "yaw",
]
stats = ["max", "median", "min", "std", "mean"]
DROPPED_FACIAL_FEATURES = [
    f"{feature}_{stat}" for feature in facial_features for stat in stats
]


DROPPED_PROSODIC_COLUMNS = []

ALREADY_NORMALIZED_FEATURES = [
    "average_outer_brow_height_mean",
    "average_inner_brow_height_mean",
    "eye_open_mean",
    "inner_lip_height_mean",
    "inner_lip_height_mean",
    "lip_corner_distance_mean",
    "average_outer_brow_height_std",
    "average_inner_brow_height_std",
    "eye_open_std",
    "outer_lip_height_std",
    "inner_lip_height_std",
    "lip_corner_distance_std",
    "average_outer_brow_height_min",
    "average_inner_brow_height_min",
    "eye_open_min",
    "outer_lip_height_min",
    "inner_lip_height_min",
    "lip_corner_distance_min",
    "average_outer_brow_height_max",
    "average_inner_brow_height_max",
    "eye_open_max",
    "outer_lip_height_max",
    "inner_lip_height_max",
    "lip_corner_distance_max",
    "average_outer_brow_height_median",
    "average_inner_brow_height_median",
    "eye_open_median",
    "outer_lip_height_median",
    "inner_lip_height_median",
    "lip_corner_distance_median",
]  # these are already in [0, 1]

MUST_KEEP_FEATURES = [
    "pause_duration_avg",
    "average_outer_brow_height_mean",
    "average_inner_brow_height_mean",
    "outer_lip_height_mean",
    "Duration/Filler Words",
]


GROUPS_COLUMN = "cleaned_ids"
INDEX_COLUMN = "participant_id"


def pearson_corr(y_true, y_pred):
    return np.corrcoef(y_true, y_pred)[0, 1]


SCORING_METRICS = {
    "r2": "r2",
    "mae": "neg_mean_absolute_error",
    "pearson": make_scorer(pearson_corr),  # Pearson Correlation Coefficient
}


MUST_KEEP_FEATURES = [
    # "pause_duration_avg",
    # "average_outer_brow_height_mean",
    # "average_inner_brow_height_mean",
    # "outer_lip_height_mean",
    "Duration/Filler Words",
]

PIPELINE_PARAMS = {'feature_selection__estimator__alpha': 0.057376790661083456, 'svr__C': 0.655379988356498, 'svr__gamma': 0.02784736494309893, 'svr__epsilon': 0.2617249201838037, 'svr__kernel': 'rbf'}
HYPERPARAMETER_TUNING_ENABLED = True

# Data Preprocessing

## Import Datasets

In [4]:
import pandas as pd
import os

features_df = pd.read_csv(os.path.join(SCRIPT_DIR, "datasets", "add.csv"))
features_df = features_df.set_index("participant_id")

labels_df = pd.read_csv(
    os.path.join(os.path.join(SCRIPT_DIR,  "datasets", "turker_scores_full_interview.csv"))
)
labels_df = labels_df.set_index("Participant")
labels_df = labels_df.loc[labels_df["Worker"] == "AGGR"]

features_df.index = features_df.index.str.lower()
labels_df.index = labels_df.index.str.lower()
indexed_combined_df = features_df.join(labels_df[[target_column]], how="left")
combined_df = indexed_combined_df.reset_index(drop=True)

# Model

## Split Data

In [5]:
X = combined_df.drop(columns=[target_column, GROUPS_COLUMN])
y = combined_df[target_column]

## Pipeline Creation

In [6]:
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import SelectFromModel
from sklearn.impute import SimpleImputer
from sklearn.linear_model import Lasso, LassoCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.model_selection import GroupKFold, cross_val_score, cross_validate
import sys

sys.path.append("..")
from models.domain_aware_selector import DomainAwareSelector

groups_column = combined_df[GROUPS_COLUMN].astype(str).values


preprocessor = ColumnTransformer(
    [
        ('dropper', 'drop', DROPPED_FACIAL_FEATURES + 
                            DROPPED_LEXICAL_COLUMNS + 
                            DROPPED_PROSODIC_COLUMNS)
    ],
    remainder='passthrough'
)
unfitted_pipeline = Pipeline(
    [
        ('preprocessor', preprocessor),
        ("imputer", SimpleImputer(strategy="mean")),  # NaN imputation
        ("scaler", StandardScaler()),
        ("feature_selection", SelectFromModel(estimator=Lasso(    max_iter=30000))),
        # ('feature_selection', DomainAwareSelector(
        #     must_keep_features=MUST_KEEP_FEATURES,
        #     selector=SelectFromModel(lasso_feature_selection_model, max_features=10),
        # )),
        ("svr", SVR(kernel="rbf")),
    ]
)

## Hyperparameter Tuning

In [7]:
import optuna
from sklearn.base import clone
from sklearn.model_selection import GroupShuffleSplit, cross_val_score
import numpy as np

def objective(trial):
    pipeline_clone = clone(unfitted_pipeline)  # Clone pipeline for thread safety
    
    params = {
        "feature_selection__estimator__alpha": trial.suggest_float(
            "feature_selection__estimator__alpha", 1e-3, 0.3, log=True
        ),
        "svr__C": trial.suggest_float("svr__C", 0.01, 100, log=True),
        "svr__gamma": trial.suggest_float("svr__gamma", 1e-3, 1e1, log=True),
        "svr__epsilon": trial.suggest_float("svr__epsilon", 0.01, 0.5),
        "svr__kernel": trial.suggest_categorical("svr__kernel", ["rbf"
                                                                #  , "poly"
                                                                 ]),
    }
    
    # if params["svr__kernel"] == "poly":
    #     params["svr__degree"] = trial.suggest_int("svr__degree", 2, 3)  # Reduced from 5
    #     params["svr__coef0"] = trial.suggest_float("svr__coef0", 0.0, 0.5)  # Narrower range
        
    pipeline_clone.set_params(**params)
    
    mc_cv_tuning = GroupShuffleSplit(n_splits=20, test_size=0.2, random_state=42)
    scores = cross_val_score(
        pipeline_clone, X, y, cv=mc_cv_tuning, groups=groups_column, n_jobs=1
    )
    return np.mean(scores)

if HYPERPARAMETER_TUNING_ENABLED:
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=100, n_jobs=-1, timeout=4*60)
    
    print("Best hyperparameters:", study.best_params)
    print(f"Best R² score: {study.best_value:.4f}")

/Users/bassel27/personal_projects/hireverse/venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-04-08 00:05:21,665] A new study created in memory with name: no-name-4de8f328-e1f6-4730-9749-2798ef686c3b


[I 2025-04-08 00:05:23,148] Trial 7 finished with value: 0.12106065630821308 and parameters: {'feature_selection__estimator__alpha': 0.23295489008266504, 'svr__C': 0.16329745183849026, 'svr__gamma': 0.10387488595782803, 'svr__epsilon': 0.014846825247926434, 'svr__kernel': 'rbf'}. Best is trial 7 with value: 0.12106065630821308.


[I 2025-04-08 00:05:23,173] Trial 6 finished with value: 0.18750320554977745 and parameters: {'feature_selection__estimator__alpha': 0.09076916174765001, 'svr__C': 1.9469368935578368, 'svr__gamma': 0.001027658428580485, 'svr__epsilon': 0.14314690513943187, 'svr__kernel': 'rbf'}. Best is trial 6 with value: 0.18750320554977745.


[I 2025-04-08 00:05:23,187] Trial 1 finished with value: -0.07145703833097625 and parameters: {'feature_selection__estimator__alpha': 0.0706257779274622, 'svr__C': 1.514041401957959, 'svr__gamma': 3.649387321031822, 'svr__epsilon': 0.043420991057115034, 'svr__kernel': 'rbf'}. Best is trial 6 with value: 0.18750320554977745.


[I 2025-04-08 00:05:23,191] Trial 4 finished with value: 0.10735348483626794 and parameters: {'feature_selection__estimator__alpha': 0.08396723195035317, 'svr__C': 0.14906047608030865, 'svr__gamma': 0.009479943915147532, 'svr__epsilon': 0.4491244026168849, 'svr__kernel': 'rbf'}. Best is trial 6 with value: 0.18750320554977745.


...

[I 2025-04-08 00:05:41,470] Trial 96 finished with value: 0.18657713811276241 and parameters: {'feature_selection__estimator__alpha': 0.14595531445201756, 'svr__C': 2.1110299188622093, 'svr__gamma': 0.022388572338239248, 'svr__epsilon': 0.1954891573133331, 'svr__kernel': 'rbf'}. Best is trial 61 with value: 0.30570881443235776.


[I 2025-04-08 00:05:41,568] Trial 97 finished with value: 0.2191371144102261 and parameters: {'feature_selection__estimator__alpha': 0.13602360217037018, 'svr__C': 2.0298388260631204, 'svr__gamma': 0.01770428709770485, 'svr__epsilon': 0.24204094467618115, 'svr__kernel': 'rbf'}. Best is trial 61 with value: 0.30570881443235776.


[I 2025-04-08 00:05:41,590] Trial 98 finished with value: 0.21626138236573972 and parameters: {'feature_selection__estimator__alpha': 0.13525917464601594, 'svr__C': 2.0439741064374664, 'svr__gamma': 0.020073381601721365, 'svr__epsilon': 0.19831354214179459, 'svr__kernel': 'rbf'}. Best is trial 61 with value: 0.30570881443235776.


[I 2025-04-08 00:05:41,596] Trial 99 finished with value: 0.16921452477697213 and parameters: {'feature_selection__estimator__alpha': 0.14848700701355116, 'svr__C': 3.296925631774624, 'svr__gamma': 0.021407455933109498, 'svr__epsilon': 0.19511011967228536, 'svr__kernel': 'rbf'}. Best is trial 61 with value: 0.30570881443235776.


Best hyperparameters: {'feature_selection__estimator__alpha': 0.07713372658882832, 'svr__C': 4.375518726867743, 'svr__gamma': 0.006825349238374233, 'svr__epsilon': 0.18432927654425854, 'svr__kernel': 'rbf'}
Best R² score: 0.3057


## Store Trained Model

In [8]:
from sklearn import clone

if HYPERPARAMETER_TUNING_ENABLED:
    unfitted_pipeline.set_params(**study.best_params)
else:
    unfitted_pipeline.set_params(**PIPELINE_PARAMS)
fitted_pipeline = clone(unfitted_pipeline)
fitted_pipeline.fit(X, y)

/Users/bassel27/personal_projects/hireverse/venv/lib/python3.13/site-packages/sklearn/compose/_column_transformer.py:1667: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('dropper', 'drop',
                                                  ['average_inner_brow_height_max',
                                                   'average_inner_brow_height_median',
                                                   'average_inner_brow_height_min',
                                                   'average_inner_brow_height_std',
                                                   'average_inner_brow_height_mean',
                                                   'average_outer_brow_height_max',
                                                   'average_outer_brow_height_median',
                                                   'average_out...
                                                   'outer_lip_height_max',
                                                   'outer_lip_height_median',
                                                   'outer_lip_height_min',
                                                   'outer_lip_height_std',
                                                   'outer_lip_height_mean', ...])])),
                ('imputer', SimpleImputer()), ('scaler', StandardScaler()),
                ('feature_selection',
                 SelectFromModel(estimator=Lasso(alpha=0.07713372658882832,
                                                 max_iter=30000))),
                ('svr',
                 SVR(C=4.375518726867743, epsilon=0.18432927654425854,
                     gamma=0.006825349238374233))])

## Feature Selection Results

In [9]:
preprocessor = fitted_pipeline.named_steps['preprocessor']
feature_names = preprocessor.get_feature_names_out()    # after preprocessing

feature_selector = fitted_pipeline.named_steps['feature_selection']
selected_mask = feature_selector.get_support()

selected_features = feature_names[selected_mask]
unselected_features = feature_names[~selected_mask]

print(f"Number of Selected features ({len(selected_features)}):")
print(f"Selected features: {selected_features}")
print(f"Unselected features: {unselected_features}")

Number of Selected features (8):
Selected features: ['remainder__intensity_mean' 'remainder__f3_sd' 'remainder__f2_f1_mean'
 'remainder__percent_unvoiced' 'remainder__percent_breaks'
 'remainder__Duration/Total Words' 'remainder__They'
 'remainder__Cognitive']
Unselected features: ['remainder__f0_mean' 'remainder__f0_min' 'remainder__f0_max'
 'remainder__f0_range' 'remainder__f0_sd' 'remainder__intensity_min'
 'remainder__intensity_max' 'remainder__intensity_range'
 'remainder__intensity_sd' 'remainder__f1_mean' 'remainder__f1_sd'
 'remainder__f2_mean' 'remainder__f2_sd' 'remainder__f3_mean'
 'remainder__f3_f1_mean' 'remainder__f2_f1_sd' 'remainder__f3_f1_sd'
 'remainder__jitter' 'remainder__shimmer' 'remainder__pause_duration_max'
 'remainder__pause_duration_avg' 'remainder__duration'
 'remainder__Total Words' 'remainder__Unique Words'
 'remainder__Filler Words' 'remainder__Audio Duration (s)'
 'remainder__Duration/Unique Words' 'remainder__Duration/Filler Words'
 'remainder__Individu

# Monte Carlo Cross Validation

In [10]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import r2_score

scoring = {
    "r2": make_scorer(r2_score),
    "pearson": make_scorer(pearson_corr)
}

results = cross_validate(
    unfitted_pipeline,
    X,
    y,
    cv=GroupShuffleSplit(n_splits=1000, test_size=0.2, random_state=42),
    groups=groups_column,
    scoring=scoring,
    n_jobs=-1,
)

r2_scores = results["test_r2"]
pearson_scores = results["test_pearson"]
# Report the average performance and variability
avg_r2_score = np.mean(r2_scores)
print(
    f"Mean R² Score: {avg_r2_score:.2f} (±{np.std(r2_scores):.2f})"
)
avg_perason_score = np.mean(pearson_scores)
print(
    f"Mean Pearson Correlation: {avg_perason_score:.2f} (±{np.std(pearson_scores):.2f})"
)

# Mean R² Score: 0.20 (±0.23)
# Mean Pearson Correlation: 0.53 (±0.16)

Traceback (most recent call last):


Traceback (most recent call last):


  File "/Users/bassel27/.vscode/extensions/ms-python.debugpy-2025.6.0-darwin-arm64/bundled/libs/debugpy/_vendored/pydevd/_pydevd_bundle/pydevd_comm.py", line 422, in _on_run
    cmd.send(self.sock)
    ~~~~~~~~^^^^^^^^^^^


  File "/Users/bassel27/.vscode/extensions/ms-python.debugpy-2025.6.0-darwin-arm64/bundled/libs/debugpy/_vendored/pydevd/_pydevd_bundle/pydevd_comm.py", line 422, in _on_run
    cmd.send(self.sock)
    ~~~~~~~~^^^^^^^^^^^


  File "/Users/bassel27/.vscode/extensions/ms-python.debugpy-2025.6.0-darwin-arm64/bundled/libs/debugpy/_vendored/pydevd/_pydevd_bundle/pydevd_net_command.py", line 111, in send
    sock.sendall(("Content-Length: %s\r\n\r\n" % len(as_bytes)).encode("ascii"))
    ~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^


...

  File "/Users/bassel27/.vscode/extensions/ms-python.debugpy-2025.6.0-darwin-arm64/bundled/libs/debugpy/_vendored/pydevd/pydevd.py", line 2558, in dispose_and_kill_all_pydevd_threads
    self._client_socket.close()
    ~~~~~~~~~~~~~~~~~~~~~~~~~^^


  File "/opt/homebrew/Cellar/python@3.13/3.13.1/Frameworks/Python.framework/Versions/3.13/lib/python3.13/socket.py", line 505, in close
    self._real_close()
    ~~~~~~~~~~~~~~~~^^


  File "/opt/homebrew/Cellar/python@3.13/3.13.1/Frameworks/Python.framework/Versions/3.13/lib/python3.13/socket.py", line 499, in _real_close
    _ss.close(self)
    ~~~~~~~~~^^^^^^


OSError: [Errno 9] Bad file descriptor


Mean R² Score: 0.20 (±0.28)
Mean Pearson Correlation: 0.54 (±0.16)


# Save the Model

In [11]:
from joblib import dump, load

dump(fitted_pipeline, os.path.join(SAVED_MODELS_PATH,f'{target_column}.joblib'))

['./regression_models/saved_models/RecommendHiring.joblib']